In [1]:
import photutils

from astropy.io import fits

from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from photutils.aperture import SkyCircularAperture, SkyCircularAnnulus, aperture_photometry

from astropy.visualization import ImageNormalize, SqrtStretch

In [7]:
def automated_photometry(clumps, wcs, science_image, mask_aperture_size=0.24, annulus_inner=0.24, annulus_outer=0.36):
    results = []

    # Convert the science image to a floating point array if not already
    science_image = np.array(science_image, dtype=float)

    # Iterates over each clump
    for index, target_clump in clumps.iterrows():
        # Check if the target clump has valid coordinates. Skips processing if the DEC value of the clump is set to a (-99.0), 
        # which indicates invalid or missing data.
        if target_clump['DEC_clump'] == -99.0:
            print(f"Skipping clump due to invalid coordinates: {target_clump['CANDELS_ID']}, {target_clump['Clump_ID']}")
            continue

        # Converts the RA and DEC of the target clump to a SkyCoord object
        target_coords = SkyCoord(ra=target_clump['RA_clump']*u.degree, dec=target_clump['DEC_clump']*u.degree)

        # Creates a copy of the original image to mask out other clumps, 
        # so that they do not affect the photometry of the target clump.
        science_image_masked = np.copy(science_image)

        # Mask other clumps only if more than one clump is present, (by setting their pixel values to 0)
        if len(clumps) > 1:
            for _, other_clump in clumps.drop(index).iterrows():
                other_coords = SkyCoord(ra=other_clump['RA_clump']*u.degree, dec=other_clump['DEC_clump']*u.degree)
                mask_aperture = SkyCircularAperture(other_coords, r=mask_aperture_size*u.arcsec).to_pixel(wcs)
                mask = mask_aperture.to_mask(method='center')
                # Generates slices for numpy array indexing using the bounding box of the mask.
                bbox_slices = (slice(mask.bbox.iymin, mask.bbox.iymax), slice(mask.bbox.ixmin, mask.bbox.ixmax))
                # Accesses the section of the image to mask out.
                science_image_masked_section = science_image_masked[bbox_slices]
                mask_data = mask.data.astype(bool)
                # Applies the mask to the image 
                science_image_masked_section[mask_data] = 0
                science_image_masked[bbox_slices] = science_image_masked_section

        # Perform photometry on the target clump
        target_aperture = SkyCircularAperture(target_coords, r=0.18*u.arcsec).to_pixel(wcs)
        photometry = aperture_photometry(science_image_masked, target_aperture)

        # Check if photometry result is unexpected!(We get this since some images in some bands are just black)
        if photometry['aperture_sum'][0] == 0:
            print(f"Zero flux detected for clump: {target_clump['CANDELS_ID']}, {target_clump['Clump_ID']}")
            

        # Background subtraction
        annulus_aperture = SkyCircularAnnulus(target_coords, r_in=annulus_inner*u.arcsec, r_out=annulus_outer*u.arcsec).to_pixel(wcs)
        background = aperture_photometry(science_image_masked, annulus_aperture)
        area_annulus = annulus_aperture.area
        # Calculates the average background flux per pixel within the annulus
        background_per_pixel = background['aperture_sum'][0] / area_annulus
        # Scales the average background to the aperture size
        total_background_light = background_per_pixel * target_aperture.area
        corrected_flux = photometry['aperture_sum'][0] - total_background_light


        # Appends the results for this clump 
        results.append({
            'CANDELS_ID': target_clump['CANDELS_ID'],
            'Clump_ID': target_clump['Clump_ID'],
            'Raw_Flux': photometry['aperture_sum'].item(),
            'Background_Flux': total_background_light,
            'Corrected_Flux': corrected_flux.item()
        })
    # Converts the results list into a DataFrame
    return pd.DataFrame(results)


In [4]:
# Load the catalog
clump_catalog_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/catalog_n/goodsslast.csv'
clump_catalog = pd.read_csv(clump_catalog_path)


In [5]:
# Initialize dictionaries
wcs_dict = {}
image_data_dict = {}

# Loop through each galaxy ID in the catalog
for candels_id in clump_catalog['CANDELS_ID'].unique():
    
    fits_path = f'/Users/neal/Documents/PhD/projects/SED-fitting/Clump/Cutouts_galaxies/goodss_{candels_id}_f435w_images.fits'
    
    # Load the FITS file
    with fits.open(fits_path) as hdul:
        # Create the WCS object from the header
        wcs_dict[candels_id] = WCS(hdul[0].header)
        # Load the science image data
        image_data_dict[candels_id] = hdul[0].data

1)

In [6]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import ImageNormalize, SqrtStretch
from photutils.aperture import SkyCircularAperture, SkyCircularAnnulus, aperture_photometry
import matplotlib.pyplot as plt

# Function to visualize the photometry process for a given clump
def visualize_photometry(target_clump, wcs, science_image, save_path, mask_aperture_size=0.24, annulus_inner=0.24, annulus_outer=0.36):
    target_coords = SkyCoord(ra=target_clump['RA_clump']*u.degree, dec=target_clump['DEC_clump']*u.degree)
    target_pixel_coords = wcs.world_to_pixel(target_coords)

    aperture = SkyCircularAperture(target_coords, r=0.18*u.arcsec).to_pixel(wcs)
    annulus = SkyCircularAnnulus(target_coords, r_in=annulus_inner*u.arcsec, r_out=annulus_outer*u.arcsec).to_pixel(wcs)

    plt.figure(figsize=(10, 10))
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(science_image, cmap='Greys', origin='lower', norm=norm)
    aperture.plot(color='blue', lw=1.5, alpha=0.5)
    annulus.plot(color='red', lw=1.5, alpha=0.5)
    plt.scatter(target_pixel_coords[0], target_pixel_coords[1], s=30, color='cyan', label='Target Clump')
    plt.xlim(target_pixel_coords[0] - 50, target_pixel_coords[0] + 50)
    plt.ylim(target_pixel_coords[1] - 50, target_pixel_coords[1] + 50)
    plt.legend()

    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()


In [8]:
import os

# List of bands 
bands = ['f125w', 'f140w', 'f160w', 'f275w', 'f435w', 'f606w', 'f775w', 'f850lp']

# Initialize a dictionary to hold the WCS objects and science image data for each band
wcs_dict = {band: {} for band in bands}
image_data_dict = {band: {} for band in bands}

# Populate the dictionaries with WCS and image data for each band and galaxy
for candels_id in clump_catalog['CANDELS_ID'].unique():
    for band in bands:
        fits_path = f'/Users/neal/Documents/PhD/projects/SED-fitting/Clump/Cutouts_galaxies/goodss_{candels_id}_{band}_images.fits'
        
        # Check if the FITS file exists
        if os.path.exists(fits_path):
            with fits.open(fits_path) as hdul:
                wcs_dict[band][candels_id] = WCS(hdul[0].header)
                image_data_dict[band][candels_id] = hdul[0].data

# Perform photometry for each band and galaxy, and add results to the catalog
for candels_id in clump_catalog['CANDELS_ID'].unique():
    galaxy_clumps = clump_catalog[(clump_catalog['CANDELS_ID'] == candels_id) & (clump_catalog['DEC_clump'] <= 90) & (clump_catalog['DEC_clump'] >= -90)].copy()
    
    for band in bands:
        # Check if the galaxy has data in this band
        if candels_id in wcs_dict[band] and candels_id in image_data_dict[band]:
            wcs = wcs_dict[band][candels_id]
            science_image = image_data_dict[band][candels_id]
            
            # Check if the science image is not empty or  black
            if not np.all(science_image == 0):
                photometry_results = automated_photometry(galaxy_clumps, wcs, science_image)
                
                # Update the original clump_catalog with the results
                for index, row in photometry_results.iterrows():
                    mask = (clump_catalog['CANDELS_ID'] == row['CANDELS_ID']) & (clump_catalog['Clump_ID'] == row['Clump_ID'])
                    clump_catalog.loc[mask, f'Raw_Flux_{band}'] = row['Raw_Flux']
                    clump_catalog.loc[mask, f'Background_Flux_{band}'] = row['Background_Flux']
                    clump_catalog.loc[mask, f'Corrected_Flux_{band}'] = row['Corrected_Flux']

# Save the updated clump catalog with photometry results
clump_catalog.to_csv('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/test_2_multi_band_photometry_results.csv', index=False)


Zero flux detected for clump: 18, 1
Zero flux detected for clump: 18, 2
Zero flux detected for clump: 172, 1
Zero flux detected for clump: 263, 1
Zero flux detected for clump: 263, 2
Zero flux detected for clump: 263, 3
Zero flux detected for clump: 3575, 1
Zero flux detected for clump: 3575, 4
Zero flux detected for clump: 3776, 1
Zero flux detected for clump: 3776, 2
Zero flux detected for clump: 4956, 1
Zero flux detected for clump: 4956, 2
Zero flux detected for clump: 5159, 1
Zero flux detected for clump: 5159, 2
Zero flux detected for clump: 5159, 3
Zero flux detected for clump: 5159, 4
Zero flux detected for clump: 6588, 1
Zero flux detected for clump: 6588, 2
Zero flux detected for clump: 6588, 3
Zero flux detected for clump: 6588, 4
Zero flux detected for clump: 6588, 5
Zero flux detected for clump: 6588, 6
Zero flux detected for clump: 6588, 7
Zero flux detected for clump: 7021, 1
Zero flux detected for clump: 7466, 1
Zero flux detected for clump: 7466, 2
Zero flux detected f

In [8]:
visualization_dir = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/Visualizations'
if not os.path.exists(visualization_dir):
    os.makedirs(visualization_dir)

for candels_id in clump_catalog['CANDELS_ID'].unique():
    for band in bands:
        if candels_id in wcs_dict[band] and candels_id in image_data_dict[band]:
            wcs = wcs_dict[band][candels_id]
            science_image = image_data_dict[band][candels_id]
            galaxy_clumps = clump_catalog[(clump_catalog['CANDELS_ID'] == candels_id)]

            for index, target_clump in galaxy_clumps.iterrows():
                save_path = os.path.join(visualization_dir, f"Galaxy_{candels_id}_Band_{band}_Clump_{target_clump['Clump_ID']}.png")
                visualize_photometry(target_clump, wcs, science_image, save_path=save_path)

/Users/neal/anaconda3/lib/python3.10/site-packages/astropy/visualization/mpl_normalize.py:179: RuntimeWarning: invalid value encountered in divide
  np.true_divide(values, self.vmax - self.vmin, out=values)


ValueError: Latitude angle(s) must be within -90 deg <= angle <= 90 deg, got -99.0 deg

getting error bcz of -99 cells in RA/Dec


In [9]:
visualization_dir = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/Visualizations'
if not os.path.exists(visualization_dir):
    os.makedirs(visualization_dir)

for candels_id in clump_catalog['CANDELS_ID'].unique():
    for band in bands:
        if candels_id in wcs_dict[band] and candels_id in image_data_dict[band]:
            wcs = wcs_dict[band][candels_id]
            science_image = image_data_dict[band][candels_id]
            galaxy_clumps = clump_catalog[clump_catalog['CANDELS_ID'] == candels_id]

            for index, target_clump in galaxy_clumps.iterrows():
                # Skip clumps with invalid DEC values
                if target_clump['DEC_clump'] == -99.0:
                    continue

                save_path = os.path.join(visualization_dir, f"Galaxy_{candels_id}_Band_{band}_Clump_{target_clump['Clump_ID']}.png")
                visualize_photometry(target_clump, wcs, science_image, save_path=save_path)


saving color pictures:

In [10]:
from astropy.visualization import ImageNormalize, SqrtStretch
from photutils.aperture import SkyCircularAperture, SkyCircularAnnulus
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt

def visualize_photometry(target_clump, wcs, science_image, save_path, mask_aperture_size=0.24, annulus_inner=0.24, annulus_outer=0.36):
    target_coords = SkyCoord(ra=target_clump['RA_clump']*u.degree, dec=target_clump['DEC_clump']*u.degree)
    target_pixel_coords = wcs.world_to_pixel(target_coords)

    aperture = SkyCircularAperture(target_coords, r=0.18*u.arcsec).to_pixel(wcs)
    annulus = SkyCircularAnnulus(target_coords, r_in=annulus_inner*u.arcsec, r_out=annulus_outer*u.arcsec).to_pixel(wcs)

    fig, ax = plt.subplots(figsize=(10, 10))
    norm = ImageNormalize(stretch=SqrtStretch())
    im = ax.imshow(science_image, cmap='Greys', origin='lower', norm=norm)
    aperture.plot(color='blue', lw=1.5, alpha=0.5, ax=ax)
    annulus.plot(color='red', lw=1.5, alpha=0.5, ax=ax)
    ax.scatter(target_pixel_coords[0], target_pixel_coords[1], s=30, color='cyan', label='Target Clump')
    ax.set_xlim(target_pixel_coords[0] - 50, target_pixel_coords[0] + 50)
    ax.set_ylim(target_pixel_coords[1] - 50, target_pixel_coords[1] + 50)
    ax.legend()

    # Create a colorbar
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Flux (arbitrary units)', rotation=270, labelpad=15)

    # Save the figure
    if save_path:
        fig.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()


In [11]:
import os
from pathlib import Path

# Base directory for visualizations
visualization_dir = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/colorVisualizations'

# Ensure the base directory exists
Path(visualization_dir).mkdir(parents=True, exist_ok=True)

for candels_id in clump_catalog['CANDELS_ID'].unique():
    for band in bands:
        if candels_id in wcs_dict[band] and candels_id in image_data_dict[band]:
            wcs = wcs_dict[band][candels_id]
            science_image = image_data_dict[band][candels_id]
            galaxy_clumps = clump_catalog[clump_catalog['CANDELS_ID'] == candels_id]
            
            # Create a subdirectory for the current band
            band_dir = os.path.join(visualization_dir, band)
            Path(band_dir).mkdir(parents=True, exist_ok=True)

            for index, target_clump in galaxy_clumps.iterrows():
                # Skip clumps with invalid DEC values
                if target_clump['DEC_clump'] == -99.0:
                    continue

                # Define the save path, now including the band subdirectory
                save_path = os.path.join(band_dir, f"Galaxy_{candels_id}_Band_{band}_Clump_{target_clump['Clump_ID']}.png")
                visualize_photometry(target_clump, wcs, science_image, save_path=save_path)


Doing color image:

In [9]:
from astropy.visualization import ImageNormalize, SqrtStretch
from photutils.aperture import SkyCircularAperture, SkyCircularAnnulus
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt

def visualize_photometry(target_clump, wcs, science_image, save_path, mask_aperture_size=0.24, annulus_inner=0.24, annulus_outer=0.36):
    target_coords = SkyCoord(ra=target_clump['RA_clump']*u.degree, dec=target_clump['DEC_clump']*u.degree)
    target_pixel_coords = wcs.world_to_pixel(target_coords)

    aperture = SkyCircularAperture(target_coords, r=0.18*u.arcsec).to_pixel(wcs)
    annulus = SkyCircularAnnulus(target_coords, r_in=annulus_inner*u.arcsec, r_out=annulus_outer*u.arcsec).to_pixel(wcs)

    fig, ax = plt.subplots(figsize=(10, 10))
    norm = ImageNormalize(stretch=SqrtStretch())
    im = ax.imshow(science_image, cmap='viridis', origin='lower', norm=norm)
    aperture.plot(color='blue', lw=1.5, alpha=0.5, ax=ax)
    annulus.plot(color='red', lw=1.5, alpha=0.5, ax=ax)
    ax.scatter(target_pixel_coords[0], target_pixel_coords[1], s=30, color='cyan', label='Target Clump')
    ax.set_xlim(target_pixel_coords[0] - 50, target_pixel_coords[0] + 50)
    ax.set_ylim(target_pixel_coords[1] - 50, target_pixel_coords[1] + 50)
    ax.legend()

    # Create a colorbar
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label('Flux (arbitrary units)', rotation=270, labelpad=15)

    # Save the figure
    if save_path:
        fig.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()


In [10]:
visualization_dir = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/viridisVisualizations'
if not os.path.exists(visualization_dir):
    os.makedirs(visualization_dir)

for candels_id in clump_catalog['CANDELS_ID'].unique():
    for band in bands:
        if candels_id in wcs_dict[band] and candels_id in image_data_dict[band]:
            wcs = wcs_dict[band][candels_id]
            science_image = image_data_dict[band][candels_id]
            galaxy_clumps = clump_catalog[clump_catalog['CANDELS_ID'] == candels_id]

            for index, target_clump in galaxy_clumps.iterrows():
                # Skip clumps with invalid DEC values
                if target_clump['DEC_clump'] == -99.0:
                    continue

                save_path = os.path.join(visualization_dir, f"Galaxy_{candels_id}_Band_{band}_Clump_{target_clump['Clump_ID']}.png")
                visualize_photometry(target_clump, wcs, science_image, save_path=save_path)



: 